In [24]:
import pandas as pd

In [35]:
cars_data = pd.read_csv('/content/sample_data/car_sales_data.csv')
cars_data['Sale_id'] = cars_data.index
cars_data.head(10)

,Date,Salesperson,Customer Name,Car Make,Car Model,Car Year,Sale Price,Commission Rate,Commission Earned,Sale_id
0,2022-08-01,Monica Moore MD,Mary Butler,Nissan,Altima,2018,15983,0.070495,1126.73,0
1,2023-03-15,Roberto Rose,Richard Pierce,Nissan,F-150,2016,38474,0.134439,5172.40,1
2,2023-04-29,Ashley Ramos,Sandra Moore,Ford,Civic,2016,33340,0.114536,3818.63,2
3,2022-09-04,Patrick Harris,Johnny Scott,Ford,Altima,2013,41937,0.092191,3866.20,3
4,2022-06-16,Eric Lopez,Vanessa Jones,Honda,Silverado,2022,20256,0.113490,2298.85,4
5,2022-12-18,Terry Perkins MD,John Olsen,Ford,Altima,2015,14769,0.077247,1140.86,5
6,2022-06-12,Ashley Brown,Tyler Lawson,Honda,F-150,2013,41397,0.142780,5910.67,6
7,2022-06-20,Norma Watkins,Michael Bond,Ford,Altima,2015,46233,0.071624,3311.38,7
8,2022-09-02,Scott Parker,Stephanie Smith,Ford,Corolla,2021,27337,0.099504,2720.13,8
9,2023-04-06,Andrew Smith,Ashley Moreno DDS,Ford,Civic,2018,16309,0.149926,2445.14,9


In [36]:
cars_data['Date'] = pd.to_datetime(cars_data['Date'])
cars_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500000 entries, 0 to 2499999
Data columns (total 10 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Date               datetime64[ns]
 1   Salesperson        object        
 2   Customer Name      object        
 3   Car Make           object        
 4   Car Model          object        
 5   Car Year           int64         
 6   Sale Price         int64         
 7   Commission Rate    float64       
 8   Commission Earned  float64       
 9   Sale_id            int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 190.7+ MB


In [37]:
dates_dim = cars_data['Date'].reset_index(drop=True).to_frame()
dates_dim['Date_id'] = dates_dim.index
dates_dim['Date'] = dates_dim['Date']
dates_dim['Hour'] = dates_dim['Date'].dt.hour
dates_dim['Day'] = dates_dim['Date'].dt.day
dates_dim['Month'] = dates_dim['Date'].dt.month
dates_dim['Year'] = dates_dim['Date'].dt.year
dates_dim = dates_dim[['Date_id', 'Date', 'Hour', 'Day', 'Month', 'Year']]
dates_dim.head(5)

,Date_id,Date,Hour,Day,Month,Year
0,0,2022-08-01,0,1,8,2022
1,1,2023-03-15,0,15,3,2023
2,2,2023-04-29,0,29,4,2023
3,3,2022-09-04,0,4,9,2022
4,4,2022-06-16,0,16,6,2022


In [38]:
Sales_dim = cars_data['Salesperson'].reset_index(drop=True).to_frame()
Sales_dim['Sale_person_id'] = Sales_dim.index
Sales_dim['Sale_person'] = Sales_dim['Salesperson']
Sales_dim['Commission_rate'] = cars_data['Commission Rate']
Sales_dim = Sales_dim[['Sale_person_id', 'Sale_person', 'Commission_rate']]
Sales_dim.head(5)

,Sale_person_id,Sale_person,Commission_rate
0,0,Monica Moore MD,0.070495
1,1,Roberto Rose,0.134439
2,2,Ashley Ramos,0.114536
3,3,Patrick Harris,0.092191
4,4,Eric Lopez,0.113490


In [39]:
Cars_dim = cars_data['Car Make'].reset_index(drop=True).to_frame()
Cars_dim['Car_id'] = Cars_dim.index
Cars_dim['Car_make'] = Cars_dim['Car Make']
Cars_dim['Car_model'] = cars_data['Car Model']
Cars_dim['Car_year'] = cars_data['Car Year']
Cars_dim = Cars_dim[['Car_id', 'Car_make', 'Car_model', 'Car_year']]
Cars_dim.head(5)

,Car_id,Car_make,Car_model,Car_year
0,0,Nissan,Altima,2018
1,1,Nissan,F-150,2016
2,2,Ford,Civic,2016
3,3,Ford,Altima,2013
4,4,Honda,Silverado,2022


In [40]:
Cars_fact = cars_data.merge(dates_dim, left_on='Sale_id', right_on='Date_id') \
                     .merge(Sales_dim, left_on='Sale_id', right_on='Sale_person_id') \
                     .merge(Cars_dim, left_on='Sale_id', right_on='Car_id') \
                     [['Sale_id', 'Date_id', 'Sale_person_id', 'Car_id', 'Customer Name', 'Sale Price', 'Commission Earned']]
Cars_fact.head(5)

,Sale_id,Date_id,Sale_person_id,Car_id,Customer Name,Sale Price,Commission Earned
0,0,0,0,0,Mary Butler,15983,1126.73
1,1,1,1,1,Richard Pierce,38474,5172.40
2,2,2,2,2,Sandra Moore,33340,3818.63
3,3,3,3,3,Johnny Scott,41937,3866.20
4,4,4,4,4,Vanessa Jones,20256,2298.85


In [41]:
dates_dim.to_csv('Dates_dim.csv')
Sales_dim.to_csv('Sales_dim.csv')
Cars_dim.to_csv('Cars_dim.csv')
Cars_fact.to_csv('Cars_fact.csv')